A notebook for making a 100-frame movie from 3D dust for Linnea as a test

Two environment variables need to be set in bash: DROPBOX_ROOT, and FFMPEG_ROOT. The DROPBOX_ROOT folder is the path to the top level directory of the dust-holgrams folder shared by Josh. E.g. in bash,

export DROPBOX_ROOT='/Users/catherinezucker/Dropbox/dust-holograms/'

You can also set the path to the FFMPEG executable:

export FFMPEG_PATH='/usr/local/bin'

Ideally, you would set these in your bash_profile profile  (now called zprofile in the latest OS operating systems). Alternatively, if you want to set one of these variables locally in this notebook, you can use:

```
import os
os.environ['DROPBOX_ROOT'] = '/Users/catherinezucker/Dropbox/dust-holograms'
os.environ['FFMPEG_ROOT'] = '/Usr/local/bin'

```

First thing we need to do is figure out how to write some json

In [15]:
import json
import numpy as np
import os
%matplotlib inline
#%matplotlib notebook
from matplotlib import pyplot as plt
from astropy import table

Josh is having some trouble with ```.zprofile``` so:

In [16]:
if (os.environ.keys() != 'DROPBOX_HOME'):
    os.environ['DROPBOX_ROOT'] = '/Users/jegpeek/Dropbox/dust-holograms'
    os.environ['FFMPEG_ROOT'] = '/Users/jegpeek/'

Using a variable called ```run_name``` to record everything we need about the run

In [17]:
def define_paths(run_name):
    if os.path.isdir("{}/{}/".format(os.environ['DROPBOX_ROOT'],run_name)) == False:
        os.mkdir("{}/{}/".format(os.environ['DROPBOX_ROOT'],run_name))
        os.mkdir("{}/{}/frames/".format(os.environ['DROPBOX_ROOT'],run_name))

This is the camera properties, which we will fix.

In [18]:
arc = False
if arc:
    ec = "szyz"
if not arc:
    ec = "rxyz"

t = True
cprops ={
    "projection": "stereographic",
    "step_size": 1,
    "max_dist": 500.0,
    "fov": 45.0,
    "x_pix": 60,
    "y_pix": 40,
    "vmax": "auto",
    "clip_mode": "tanh",
    "fuzzy": t,
    "randomize_angles": t,
    "euler_convention": ec}

In [19]:
print(cprops)

{'projection': 'stereographic', 'step_size': 1, 'max_dist': 500.0, 'fov': 45.0, 'x_pix': 60, 'y_pix': 40, 'vmax': 'auto', 'clip_mode': 'tanh', 'fuzzy': True, 'randomize_angles': True, 'euler_convention': 'rxyz'}


_In arc mode_ Linnea asked for 100 frames orbiting around a point with equal distance. We'll assume that the middle of these 100 frames is the Sun, and we'll set some sweep out angle in the Galactic Plane.

In [20]:
nframes = 19 # 100 frames
if arc:
    angle_sweep = 45 # half of this CCW and half CW from the sun

In [21]:
def sweep(xc, yc, angle_sweep, nframes):
    R = np.sqrt(xc**2+ yc**2)
    phi =np.arctan2(yc, xc)*180/np.pi
    dangs = np.linspace(0, angle_sweep, nframes)
    xs = xc - R*np.cos((phi-angle_sweep/2+dangs)*np.pi/180)
    ys = yc - R*np.sin((phi-angle_sweep/2+dangs)*np.pi/180)
    angs = (dangs-angle_sweep/2+phi)*np.pi/180 ## wait are these in RADIANs??
    return xs, ys, angs

In [22]:
def plot_ang(x0, y0, xs, ys, run_name):
    plt.figure(figsize=[5, 5])
    plt.scatter(x0, y0)
    plt.scatter(0, 0, c='red')
    plt.plot(xs, ys)
    plt.xlim([-500, 500])
    plt.ylim([-500, 500])
    plt.title(run_name)
    plt.savefig('{}/{}/arc_{}'.format(os.environ['DROPBOX_ROOT'],run_name,run_name))

In track mode we'll move from one point to another while pointing at the cloud and keeping to top of the camera perpindicular to the plane that contains the track and cloud

Given the current position of the viewer, the unit direction toward the start of the track, and the cloud position, compute the angles of rotation

In [23]:
def determine_angles(v_pos, c_pos, t_hat):
    x_hat = np.array([1, 0, 0])
    y_hat = np.array([0, 1, 0])
    z_hat = np.array([0, 0, 1])
    # the vector to the cloud
    c_vec = c_pos-v_pos
    # the unit vector toward the cloud
    c_hat = c_vec/np.sqrt(np.sum(c_vec**2))
    
    #FIRST
    # the unit vector perp to the plane containing the track and the cloud
    n_OCT = np.cross(c_hat, t_hat) #CHECK ORDER ####FLIPPED
    n_hat_OCT = n_OCT/np.sqrt(np.sum(n_OCT**2))
    #print('n_hat_OCT', n_hat_OCT)
    #print('c_hat', c_hat)
    #print('v_pos', v_pos)
    #print('c_vec', c_vec)
    #print('c_pos', c_pos)
    # find the new orientation of the side of the camera
    v_y_prime = np.cross(n_hat_OCT, x_hat) #CHECK ORDER (correct)
    v_hat_y_prime = v_y_prime/np.sqrt(np.sum(v_y_prime**2))
    # and find the angle to rotate first
    theta = np.arccos(np.dot(v_hat_y_prime,y_hat))
    #print('theta', theta)
    #print('v_hat_y_prime',v_hat_y_prime)
    
    #SECOND
    # the unit vector perp to the plane that contains the rotation 
    #n_phi = np.cross(v_hat_y_prime, x_hat) #CHECK ORDER ###FLIPPED
    #n_hat_phi = n_phi/np.sqrt(np.sum(n_phi**2))
    # the vector being rotated to
    #v_x_prime_prime = np.cross(n_hat_OCT,n_hat_phi) #CHECK ORDER (correct)
    #### OOOPS
    v_x_prime_prime = np.cross(v_hat_y_prime, n_hat_OCT)
    v_hat_x_prime_prime = v_x_prime_prime/np.sqrt(np.sum(v_x_prime_prime**2))
    # and the angle rotated
    phi = np.arccos(np.dot(v_hat_x_prime_prime, x_hat))
    #print('n_phi', n_phi)
    
    #FINALLY
    zeta = np.arccos(np.dot(v_hat_x_prime_prime, c_hat))
    print('c hat', c_hat)
    return theta, phi, zeta
    

given some inputs build a track

In [24]:
def track(xt1, yt1, zt1, xt2, yt2, zt2, nframes):
    xts = np.linspace(xt1, xt2, nframes)
    yts = np.linspace(yt1, yt2, nframes)
    zts = np.linspace(zt1, zt2, nframes)
    t_vec = np.array([xt2-xt1, yt2-yt1, zt2-zt1])
    t_hat = t_vec/np.sqrt(np.sum(t_vec**2))
    return xts, yts, zts, t_hat

In [25]:
def build_fprops(fprops, cprops, angs, xs, ys, zc):
    for i in range(nframes):
        fprops.append({
          "xyz": [xs[i], ys[i], zc],
          "angles": [angs[i], 0.0, 0.0]
        })
    allprops = {"camera_props": cprops,"frame_props":fprops }
    return allprops

In [26]:
def build_fprops_track(fprops, cprops, theatas, phis, zetas, xs, ys, zs):
    for i in range(nframes):
        fprops.append({
          "xyz": [xs[i], ys[i], zs[i]],
          "angles": [thetas[i], phis[i], zetas[i]]
        })
    allprops = {"camera_props": cprops,"frame_props":fprops }
    return allprops

Let's read in a list of molecular clouds and make movies for each one:

In [27]:
clouds = table.Table.read('{}/Holo_Cloud_Targets.csv'.format(os.environ['DROPBOX_ROOT']))
#clouds = table.Table.read('{}/Holo_TEST_Targets.csv'.format(os.environ['DROPBOX_ROOT']))

In [28]:
makeimages = True
makemovies = True
oldzeta = 0
for c in clouds[0:4]:
    print(c['cloud'])
    run_name = c['cloud'] + '_test_track_2'
    define_paths(run_name)
    if arc:
        xs, ys, angs = sweep(c['x'], c['y'], angle_sweep, nframes)
        print(xs, ys, angs)
        plot_ang(c['x'], c['y'], xs, ys, run_name)
        # we make an empty list of frames to which we can append frames
        fprops = []
        allprops = build_fprops(fprops, cprops, angs, xs, ys, np.float(c['z']))
    if not arc:
        #hardcoding a track
        #xts, yts, zts, t_hat = track(c['xt1'], c['yt1'], c['zt1'], c['xt2'], c['yt2'], c['zt2'], nframes)
        xts, yts, zts, t_hat = track(c['y']*0.5, c['x']*0.5, 0, c['y']*(-0.5), c['x']*(-0.5), 0, nframes)
        
        print(t_hat)
        fprops = []
        thetas = np.zeros(nframes)
        phis = np.zeros(nframes)
        zetas = np.zeros(nframes)
        for j in np.arange(nframes):
            v_pos = np.array([xts[j], yts[j], zts[j]])
            c_pos = np.array([c['x'], c['y'], c['z']])
            thetas[j], phis[j], zetas[j] = determine_angles(v_pos, c_pos, t_hat)
            print(thetas[j]*180/np.pi, phis[j]*180/np.pi, zetas[j]*180/np.pi,(zetas[j]-oldzeta)*180/np.pi )
            oldzeta = zetas[j]
        allprops = build_fprops_track(fprops, cprops, thetas, phis, zetas, xts, yts, zts)

    with open('{}/{}/{}.json'.format(os.environ['DROPBOX_ROOT'],run_name,run_name), 'w') as outfile:
        json.dump(allprops, outfile,indent=2)
    if makeimages:
        os.system("python3 project_frames.py {}/{}/{}.json {}/leike2020_bayestar19_splice.npy {}/{}/frames/{}_{{:05d}}.png"
          .format(os.environ['DROPBOX_ROOT'],run_name,run_name,os.environ['DROPBOX_ROOT'],os.environ['DROPBOX_ROOT'],run_name, run_name))
    if makemovies:
        os.system("{}/ffmpeg -y -r 30 -start_number 0 -i {}/{}/frames/{}_%05d.png -c:v libx264 -s 600x400 -r 30 -pix_fmt yuv420p {}/{}/{}.mp4"
          .format(os.environ['FFMPEG_ROOT'],os.environ['DROPBOX_ROOT'],run_name,run_name,os.environ['DROPBOX_ROOT'],run_name, run_name))


Chamaeleon
[ 0.87415728 -0.48564293  0.        ]
c hat [ 0.62485057 -0.75639806 -0.19345219]
25.299589809325603 13.355740562576404 50.042466908587876 50.042466908587876
c hat [ 0.61295617 -0.76430337 -0.20031247]
25.299589809325603 13.355740562576404 50.95027419762971 0.9078072890418311
c hat [ 0.60001952 -0.77257416 -0.20761921]
25.299589809325603 13.355740562576404 51.92456884864203 0.9742946510123224
c hat [ 0.58591426 -0.78121902 -0.21540966]
25.299589809325603 13.355740562576404 52.97230219059096 1.047733341948931
c hat [ 0.57049558 -0.79024203 -0.22372376]
25.299589809325603 13.355740562576404 54.10131594420795 1.1290137536169835
c hat [ 0.55359742 -0.79964072 -0.23260396]
25.29958980932562 13.355740562576404 55.32046227655706 1.2191463323491114
c hat [ 0.53502933 -0.80940334 -0.24209472]
25.29958980932562 13.355740562576404 56.6397351963399 1.3192729197828437
c hat [ 0.51457302 -0.81950518 -0.25224168]
25.299589809325603 13.355740562576404 58.07041070830708 1.43067551196718
c ha

100% (19 of 19) |########################| Elapsed Time: 0:00:04 ETA:  00:00:00

All workers done.
0.397824734759316
Loading map ...
Ray-casting frames ...
frame 0: vmax = 0.016659770933765687
frame 1: vmax = 0.017916911780594093
frame 2: vmax = 0.017033658425880276
frame 3: vmax = 0.018947241715497512
frame 4: vmax = 0.017468555059887876
frame 5: vmax = 0.015294391357223503
frame 6: vmax = 0.02113461638539957
frame 7: vmax = 0.016840029650498765
frame 8: vmax = 0.026146960495192616
frame 9: vmax = 0.02386520939453476
frame 10: vmax = 0.01768711079390778
frame 11: vmax = 0.02742641761469713
frame 12: vmax = 0.03112779617108754
frame 13: vmax = 0.022099464467639336
frame 14: vmax = 0.031095473719062284
frame 15: vmax = 0.023699611506206565
frame 16: vmax = 0.01926198616255715
frame 17: vmax = 0.01967479124160309
frame 18: vmax = 0.011943642129073851
Worker finished.


ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

Ophiuchus
[ 0.07728751 -0.99700885  0.        ]
c hat [ 0.84702891 -0.47092279  0.24652334]
178.64934316482677 16.912880738655456 27.70899823066707 -58.17050614102316
c hat [ 0.86386381 -0.43588798  0.25246977]
178.64934316482677 16.912880738655456 25.455874365086974 -2.253123865580099
c hat [ 0.88006881 -0.39845925  0.25828107]
178.64934316482677 16.912880738655456 23.095230166119197 -2.3606441989677736
c hat [ 0.89541001 -0.35861509  0.26388658]
178.64934316482677 16.912880738655456 20.627771884024718 -2.467458282094484
c hat [ 0.90963298 -0.31637741  0.26920841]
178.64934316482677 16.912880738655456 18.055851703691328 -2.5719201803333878
c hat [ 0.92246878 -0.27181976  0.27416303]
178.64934316482706 16.912880738655456 15.383702700159589 -2.6721490035317372
c hat [ 0.93364235 -0.22507449  0.27866366]
178.64934316482677 16.912880738655456 12.617629650204636 -2.766073049954954
c hat [ 0.94288334 -0.1763377   0.28262344]
178.64934316482677 16.912880738655456 9.766128714650328 -2.8515009

...

ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay

Taurus
[-0.12663189  0.99194978  0.        ]
c hat [-0.84045484  0.49586836 -0.21851826]
177.94424625705352 15.695210434490745 150.8097877572202 138.99091637853667
c hat [-0.85698571  0.46396094 -0.2243117 ]
177.94424625705352 15.695210434490745 152.8954350603871 2.0856473031668705
c hat [-0.87316478  0.42969934 -0.23009072]
177.94424625705352 15.695210434490745 155.09158846789438 2.196153407507317
c hat [-0.88878962  0.39300221 -0.23580133]
177.94424625705352 15.695210434490745 157.400292082505 2.308703614610585
c hat [-0.90363028  0.35381871 -0.24138069]
177.94424625705352 15.695210434490745 159.82234390477308 2.4220518222680845
c hat [-0.91743165  0.31213766 -0.24675748]
177.94424625705352 15.695210434490745 162.3570045044864 2.5346605997133547
c hat [-0.92991809  0.26799723 -0.25185281]
177.94424625705352 15.695210434490745 165.00170602155524 2.644701517068832
c hat [-0.94080078  0.22149389 -0.25658203]
177.94424625705352 15.695210434490745 167.75178470405845 2.750078682503204
c ha

100% (19 of 19) |########################| Elapsed Time: 0:00:05 ETA:  00:00:00

All workers done.
0.397824734759316
Loading map ...
Ray-casting frames ...
frame 0: vmax = 0.011659327849600231
frame 1: vmax = 0.01490865574820782
frame 2: vmax = 0.01166055076941484
frame 3: vmax = 0.015311975208158402
frame 4: vmax = 0.018385176525163843
frame 5: vmax = 0.016212674459824484
frame 6: vmax = 0.013112980787736888
frame 7: vmax = 0.015113844896670343
frame 8: vmax = 0.014957770682916817
frame 9: vmax = 0.013637898490404041
frame 10: vmax = 0.01476089895021505
frame 11: vmax = 0.021751435977328582
frame 12: vmax = 0.017710748551946383
frame 13: vmax = 0.02022713482791005
frame 14: vmax = 0.014120516257344206
frame 15: vmax = 0.014920430658450641
frame 16: vmax = 0.01368677093804763
frame 17: vmax = 0.0148204499443018
frame 18: vmax = 0.017752781319628413
Worker finished.


ffmpeg version 5.0-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 11.0.0 (clang-1100.0.33.17)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --enable-version3 --pkg-config-flags=--static --disable-ffplay